In [ ]:
import pandas as pd
import requests
import os
from datetime import datetime

FRED_API_KEY = os.getenv('FRED_API_KEY')
BASE_URL = 'https://api.stlouisfed.org/fred'

series_list = ['CPIAUCSL', 'CPILFESL', 'CPIUFDSL', 'CPIHOSSL', 
                'CUSR0000SETB01', 'PCEPI', 'PCEPILFE', 
                'DPCERD3A086NBEA', 'GASREGW', 'DCOILWTICO']

series_id = 'CPIAUCSL'

In [ ]:
# 1. 시계열 데이터 다운로드
obs_url = f'{BASE_URL}/series/observations'
obs_params = {
    'api_key': FRED_API_KEY,
    'file_type': 'json',
    'series_id': series_id,
    'observation_start': start_date,
    'observation_end': datetime.now().strftime('%Y-%m-%d')
}
obs_data = requests.get(obs_url, params=obs_params).json()
obs_df = pd.DataFrame(obs_data['observations'])
obs_df = obs_df[obs_df['value'] != '.']
obs_df['date'] = pd.to_datetime(obs_df['date'])
obs_df['value'] = pd.to_numeric(obs_df['value'])

In [ ]:
    # 2. release_id 추출 및 release_dates 수집
    release_url = f'{BASE_URL}/series/release'
    release_params = {'api_key': FRED_API_KEY, 'file_type': 'json', 'series_id': series_id}
    release_id = requests.get(release_url, params=release_params).json()['releases'][0]['id']
    rel_url = f'{BASE_URL}/release/dates'
    rel_params = {'api_key': FRED_API_KEY, 'file_type': 'json', 'release_id': release_id}
    rel_data = requests.get(rel_url, params=rel_params).json()['release_dates']
    rel_df = pd.DataFrame(rel_data)
    rel_df['release_date'] = pd.to_datetime(rel_df['date'])
    rel_df = rel_df.sort_values('release_date')

In [ ]:
if __name__ == "__main__":

    
    final_df = pd.DataFrame()
    for series in series_list:
        print(f"Processing {series}...")
        df = download_fred_series(series)
        if final_df.empty:
            final_df = df
        else:
            final_df = final_df.join(df, how='outer')
    
    final_df.index.name = 'date'
    final_df.to_csv('../data/fred_data.csv')
    print(final_df.tail())